<img style="float: right;" src="./images/DataReading.png" width="120"/>

# Reading Data

* Python has a large number of different ways to read data from external files. 
* Python supports almost any type of file you can think of, from simple text files to complex binary formats.
* In this class we are going to mainly use the package **`pandas`** to load external files into `DataFrames`.
* Most of our datafiles will be `csv` files (comma separated values)

In [ ]:
import pandas as pd
import numpy as np

##### Let us read-in the file: `./Data/Planets.csv`

```
Name,a,
Mercury,0.3871,0.2056
Earth,0.9991,0.0166
Jupiter,5.2016,0.0490
Neptune,29.9769,0.0088
```

#### Pandas assumes the first row is a list of the column names

In [ ]:
planet_table = pd.read_csv('./Data/Planets.csv')

In [ ]:
planet_table

## Renaming columns

In [ ]:
planet_table.rename(columns={'Unnamed: 2': 'ecc'}, inplace=True)

planet_table

In [ ]:
planet_table['ecc']

## Adding a column - `insert`

`.insert(loc, column, value, allow_duplicates = False)`

#### perihelion distance [AU] = `semi_major axis * ( 1 - eccentricity )`

In [ ]:
def find_perihelion(semi_major, eccentricity):
    result = semi_major * (1.0 - eccentricity)
    return result

#### Use `DataFrame` columns as arguments to the `find_perihelion` function

In [ ]:
my_perihelion = find_perihelion(planet_table['a'], planet_table['ecc'])

In [ ]:
my_perihelion

In [ ]:
# Add column in position 1 (2nd column)

planet_table.insert(1, 'Perihelion', my_perihelion, allow_duplicates = False)

In [ ]:
planet_table

## Removing a column - `drop`

In [ ]:
planet_table.drop(columns='Perihelion', inplace = True)

In [ ]:
planet_table

## Adding a column (quick) - always to the end of the table

In [ ]:
planet_table['Perihelion'] = my_perihelion

In [ ]:
planet_table

## Rearranging columns

In [ ]:
planet_table.columns

In [ ]:
my_new_order = ['a', 'Perihelion', 'Name', 'ecc']

In [ ]:
planet_table = planet_table[my_new_order]

In [ ]:
planet_table

## Adding a row `.append`

* The new row has to be a `dictionary` or another `DataFrame`
* Almost always need to use: `ignore_index=True`

In [ ]:
my_new_row = {'Name': 'Venus',
              'a': 0.723,
              'ecc': 0.007}

In [ ]:
my_new_row

In [ ]:
planet_table.append(my_new_row, ignore_index=True)

In [ ]:
planet_table

In [ ]:
planet_table = planet_table.append(my_new_row, ignore_index=True)

In [ ]:
planet_table

#### `NaN` = Not_A_Number, python's null value

----

<img style="float: right;" src="./images/Lore.jpg" width="200"/>

# Reading (bad) Data

## Different Delimiters

Some people just want to watch the world burn, so they create datasets where the columns are separted by something other than a comma.

#### Bad - Using another delimiter like `:`

##### `./Data/Planets_Ver2.txt`

```
Name:a:
Mercury:0.3871:0.2056
Earth:0.9991:0.0166
Jupiter:5.2016:0.0490
Neptune:29.9769:0.0088
```

In [ ]:
planet_table_2 = pd.read_csv('./Data/Planets_Ver2.txt',
                             delimiter = ":")

In [ ]:
planet_table_2

#### Worse - Using whitespace as a delimiter

##### `./Data/Planets_Ver3.txt`

```
Name a 
Mercury 0.3871 0.2056
Earth 0.9991 0.0166
Jupiter 5.2016 0.0490
Neptune 29.9769 0.0088
```

In [ ]:
planet_table_3 = pd.read_csv('./Data/Planets_Ver3.txt',
                             delimiter = " ")

In [ ]:
planet_table_3

#### WORST! - Using inconsistent whitespace as a delimiter!

##### `./Data/Planets_Ver4.txt`

```
 Name   a 
    Mercury 0.3871  0.2056
 Earth 0.9991   0.0166
     Jupiter 5.2016  0.0490
 Neptune    29.9769    0.0088
```

In [ ]:
planet_table_4 = pd.read_csv('./Data/Planets_Ver4.txt',
                             delimiter = " ",
                             skipinitialspace=True)

In [ ]:
planet_table_4

---

<img style="float: right;" src="./images/MessyData.jpg" width="230"/>

# Messy Data

* `pandas` is a good choice when working with messy data files.
* In the "real world" all data is messy.

##### Let us read-in the file: `./Data/Messy.csv`

```
#######################################################
#
# Col 1 - Name
# Col 2 - Size (km)
#
#######################################################
"Sample 1",10
"",23
,
# A random comment row just because
"Another Sample",
```

### This is not going to end well ... (errors galore!)

In [ ]:
messy_table = pd.read_csv('./Data/Messy.csv')

### Tell `pandas` about the comments:

In [ ]:
messy_table = pd.read_csv('./Data/Messy.csv', 
                          comment = "#")

messy_table

## Not quite correct ...

### Turn off the header

In [ ]:
messy_table = pd.read_csv('./Data/Messy.csv',
                          comment = "#",
                          header= None)

messy_table

### Add the column names

In [ ]:
my_column_name = ['Name', 'Size']

In [ ]:
messy_table = pd.read_csv('./Data/Messy.csv',
                          comment = "#",
                          header= None,
                          names = my_column_name)

messy_table

### Deal with the missing data with `.fillna()`

In [ ]:
messy_table['Name'].fillna("unknown", inplace=True)
messy_table['Size'].fillna(999.0, inplace=True)

messy_table

---
### Another way to deal with messy data is using `skiprows`

<img style="float: right;" src="./images/DataHate.gif" width="400"/>

##### Maybe someone hates you and sends you this file: `./Data/RealMessy.csv`

```
% Col 1 - Name
% Col 2 - Size (km)
"Sample 1",10
"",23
,
# A random comment row just because
"Another Sample",
```

### We want to ignore rows 0,1, and 5

In [ ]:
messy_table = pd.read_csv('./Data/ReallyMessy.csv',
                          skiprows = [0,1,5],
                          header= None,
                          names = my_column_name)

messy_table

----

# Fixed-Width Data Tables - `pd.read_fwf()`

* These types of data tables are **VERY** common in astronomy
* The columns have a fixed-widths
* Whitespace is used to seperate columns **AND** used within columns


#### Trying to read this in with `pd.read_csv()` is a mess!

In [ ]:
standard_table = pd.read_csv('./Data/StdStars.dat',
                             header= None,
                             delimiter = " ",
                             skipinitialspace=True)

In [ ]:
standard_table

### You can set the width of the columns: `widths`

<img src="./images/Width.png" width="400"/>

#### pandas will trim off surrounding whitespace

In [ ]:
my_column_widths = [12, 3, 3, 2]

In [ ]:
my_column_name = ['Star', 'RAh', 'RAm', 'RAs']

In [ ]:
standard_table = pd.read_fwf('./Data/StdStars.dat', 
                             header = None, 
                             widths = my_column_widths, 
                             names = my_column_name)

In [ ]:
standard_table

### You can set the beginning to end of a column: `colspecs`

<img src="./images/Colspec.png" width="400"/>

`colspecs` uses the same format as a slice

* 0-based indexing
* First number is the first element you want
* Second number is the first element you DON'T want

In [ ]:
my_colspecs = [(0,12), (12,14), (15,17), (18,20)]

In [ ]:
standard_table = pd.read_fwf('./Data/StdStars.dat',
                             colspecs = my_colspecs,
                             header= None,
                             names = my_column_name)

In [ ]:
standard_table

----
# Real World Example

![AJ](./images/StdStars.png)

[Landolt Paper SIMBAD page](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=1992AJ....104..340L) - 
[(Mirror)](http://simbad.cfa.harvard.edu/simbad/sim-basic?Ident=1992AJ....104..340L)

#### Set the column names

In [ ]:
my_column_name = ['Star', 
                  'RAh', 'RAm','RAs',
                  'DEd', 'DEm','DEs',
                  'Vmag', 'B-V', 'U-B', 'V-R', 'R-I', 'V-I',
                  'o_Vmag', 'Nd',
                  'e_Vmag', 'e_B-V', 'e_U-B', 'e_V-R', 'e_R-I', 'e_V-I']

#### Set the column widths

In [ ]:
my_column_width = [12, 2, 3, 3, 4, 3, 3, 8, 7, 7, 7, 7, 7, 5, 4, 8, 7, 7, 7, 7, 7]

#### Set the URL for the data

In [ ]:
my_url = 'https://cdsarc.unistra.fr/ftp/II/183A/table2.dat'

#my_url = 'Data/table2.dat'

In [ ]:
standard_table = pd.read_fwf(my_url,
                             header=None,
                             widths=my_column_width,
                             names = my_column_name)

In [ ]:
standard_table.head(10)

----

<img style="float: right;" src="./images/LotsData.jpg" width="230"/>

# Lots of Data

* `pandas` will cutoff the display of really long tables
* You can change this with:
    * `pd.set_option('display.max_rows', # of rows)`
    * `pd.set_option('display.max_columns', # of columns)`

In [ ]:
standard_table.info()

In [ ]:
pd.set_option('display.max_columns', 21)

In [ ]:
standard_table.head(10)

In [ ]:
pd.set_option('display.max_rows', 526)

In [ ]:
standard_table

----

# Bonus Content: Reading HTML tables (Wikipedia)

* `pandas` can (sort-of) easily import HTML tables - `read_html()`
* This is great for pulling in data from Wikipedia
* The results are often far from perfect

# [List of impact craters in North America](https://en.wikipedia.org/wiki/List_of_impact_craters_in_North_America)

* There are 4 tables on this page
* Plus a bunch of other table-ish-looking content
* Let us see how this works out ....

In [ ]:
crater_wiki = 'https://en.wikipedia.org/wiki/List_of_impact_craters_in_North_America'

In [ ]:
crater_table = pd.read_html(crater_wiki)

### What did we get?

In [ ]:
type(crater_table)

### A list, close but not a `pandas` DataFrame

In [ ]:
len(crater_table)

### And 7 of them. OK, what do we have at index 0?

In [ ]:
crater_table[0]

### Garbage

In [ ]:
type(crater_table[0])

### ... But is it a DataFrame!

### And at index 1?

In [ ]:
crater_table[1]

### Sweet! A DataFrame of the first table (Impact Craters in Canada)

### Index 2 is the Mexican table

In [ ]:
crater_table[2]

### Index 3 is the US Table

In [ ]:
crater_table[3]

### Index 4 is the Unconfirmed impact craters

In [ ]:
crater_table[4]

### The last two [5 and 6] are garbage

### The DataFrames are not perfect, and will need some cleaning, but is great starting point